# signal analysis notebook(Linux)
このノートブックでは、実際に筆者が書いた関数を呼び出し、それに基づいて信号の可視化を解析・解説しています。ソースコードは同じレポジトリ（src/utils.py）などを参照してください。
このファイルは、Linux向けに動作テストを実施しています。Windows環境では、別ファイルを参照してください。

## メタデータ表示
生のファイルの中にどのような情報があるのかを出力します。特に、これらはメタデータを表示するスクリプトとして利用してください。実際にデータが取得された年度において処理が異なることに留意してください。

In [1]:
from src import analyze_mat_file,analyze_mat_file_h5py
file_path_exp = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20240726-1600.mat"
file_path_sim = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
analyze_mat_file(file_path_exp)
analyze_mat_file_h5py(file_path_sim)

Analyzing: /home/smatsubara/documents/airlift/data/experiments/rawsignal/P20240726-1600.mat
File size: 3973.64 MB
Successfully loaded .mat file

=== File Metadata ===
__header__: b'PicoScope 7 T&M, Created on: 2025?5?8?'
__version__: 1.0
__globals__: []

=== Variables ===

Variable 1: Tstart
  Type: <class 'numpy.ndarray'>
  Shape: (1, 1)
  Data type: float32
  Min: 0.0
  Max: 0.0
  Mean: 0.0

Variable 2: Tinterval
  Type: <class 'numpy.ndarray'>
  Shape: (1, 1)
  Data type: float64
  Min: 1.9199999812258284e-08
  Max: 1.9199999812258284e-08
  Mean: 1.9199999812258284e-08

Variable 3: ExtraSamples
  Type: <class 'numpy.ndarray'>
  Shape: (1, 1)
  Data type: int32
  Min: 0
  Max: 0
  Mean: 0.0

Variable 4: RequestedLength
  Type: <class 'numpy.ndarray'>
  Shape: (1, 1)
  Data type: int32
  Min: 260416669
  Max: 260416669
  Mean: 260416669.0

Variable 5: Length
  Type: <class 'numpy.ndarray'>
  Shape: (1, 1)
  Data type: int32
  Min: 260416669
  Max: 260416669
  Mean: 260416669.0

Variab

/home/smatsubara/documents/airlift/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


  Mean: nan

Variable 8: TDX2
  Type: <class 'numpy.ndarray'>
  Shape: (1, 260416673)
  Data type: float32
  Min: -inf
  Max: inf
  Mean: nan

Variable 9: TDX3
  Type: <class 'numpy.ndarray'>
  Shape: (1, 260416673)
  Data type: float32
  Min: -2.3972601890563965
  Max: 2.152641773223877
  Mean: 0.004267964977771044

Variable 10: TDX1_enlarged
  Type: <class 'numpy.ndarray'>
  Shape: (1, 260416673)
  Data type: float32
  Min: -inf
  Max: inf
  Mean: nan

Total variables: 10

=== Nested Structures ===

Structure of: Tstart
Array: shape=(1, 1), dtype=float32

Structure of: Tinterval
Array: shape=(1, 1), dtype=float64

Structure of: ExtraSamples
Array: shape=(1, 1), dtype=int32

Structure of: RequestedLength
Array: shape=(1, 1), dtype=int32

Structure of: Length
Array: shape=(1, 1), dtype=int32

Structure of: Version
Array: shape=(1,), dtype=<U11

Structure of: TDX1
Array: shape=(1, 260416673), dtype=float32

Structure of: TDX2
Array: shape=(1, 260416673), dtype=float32

Structure of: TDX

## ファイル形式変換

もシミュレーション生成のファイルも`.mat`形式で保存されていますが、これらはサンプリングレートや単位が異なるほか、実機データでは15000回の測定が連続的に記録されており、境目を表示することが困難であることが多いです。  
ゆえに、処理の統一化を目的として一度すべて`.npz`形式に変換します。これによって、一つのスクリプトで統一的に信号波形を処理することができるようになります。また、特定の状況下で機械学習用のデータセットを作成しやすくなるという狙いがあります。  
以下のセルは、実機データを変換するスクリプトです。   
### 実機データ
命名規則:
 - 2024: dates and time  
    e.g. P20241007-1013.mat
 - 2023: experimental settings(requires details)  
    e.g. s0_g2_l2_t1.mat

これらに対し、`.npz`形式に変換したものを`*_processed.npz`として新しい名前を与え保存しています。
なお、`signal_key="TDX1"`と書いてありますが、これはTDX1のみを処理しているのではなく、これらをベースにトリガを検出し、その時刻を基準点としてすべてのチャンネルの信号波形を並び変えているということに注意してください。



In [2]:
from src import convert_mat2npy
import numpy as np
import os
import scipy.io as sio
file_path = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20241007-1013.mat"
output_dir = "/home/smatsubara/documents/airlift/data/experiments/processed"  # Change to your desired directory
processed_data,fs=convert_mat2npy(file_path,start_time=0.0,duration=5.0,amplitude_threshold=2,window_width=0.1e-3,signal_key="TDX1")
# If you want to save fs and other metadata together with processed_data, it is common to use a dictionary and save it as a .npz file.
# You should modify convert_mat2npy to return fs and any other metadata you need.
# Here, as an example, we use a dummy value for fs. In practice, get fs from the return value of convert_mat2npy.
mat_data=sio.loadmat(file_path)
keys=list(mat_data.keys())
print(fs)
print(keys)
# Prepare a dictionary to save both data and metadata
save_dict = {
    "processed_data": processed_data,
    "fs": fs,
    "original_keys": keys,
    # Add other metadata here if needed
}

base_filename = os.path.splitext(os.path.basename(file_path))[0]
save_path = os.path.join(output_dir, f"{base_filename}_processed.npz")
np.savez(save_path, **save_dict)
print(f"Processed data and metadata saved to: {save_path}")

Loading data...
Loading successful
Using device: cuda
Number of detected triggers: (14988,)
triggers.shape: (14988,)
signal_chunk.shape: (260416669,)
fs: 52083333.842615336
arranged_pulses_tdx1.shape: (14988, 5208, 1)
arranged_pulses_tdx1_enlarged.shape: (14988, 5208, 1)
arranged_pulses.shape: (14988, 5208, 4, 1)
52083333.842615336
['__header__', '__version__', '__globals__', 'Tstart', 'Tinterval', 'ExtraSamples', 'RequestedLength', 'Length', 'Version', 'TDX1', 'TDX2', 'TDX3', 'TDX1_enlarged']
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz


### シミュレーションデータ  
次に、シミュレーションデータを変換します。シミュレーションにおいては、特に3次元計算の場合１パルス分ですら３～４時間ほどの計算時間を要します。そのため、複数パルス分のデータを用意することはできません。  
そのため、`.npz`のファイルには１パルス分の情報だけが記録されることになります。それゆえ、それらを画像として可視化する場合、`full=True`と`full=False`の処理は同じになります。  
以下のセルはシミュレーションデータを変換するスクリプトです。


In [6]:
from src import convert_mat2npy
import numpy as np
import os
import scipy.io as sio
import h5py
file_path = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
output_dir = "/home/smatsubara/documents/airlift/data/simulation/processed"  # Change to your desired directory
#processed_data,fs=convert_mat2npy(file_path,start_time=0.0,duration=5.0,amplitude_threshold=2,window_width=0.1e-3,signal_key="TDX1")

mat_data=h5py.File(file_path, 'r')
keys=list(mat_data.keys())
#print(fs)
print(keys)
print(mat_data) 
print(mat_data.kgrid)
# Prepare a dictionary to save both data and metadata
save_dict = {
    "processed_data": processed_data,
    "fs": fs,
    "original_keys": keys,
    # Add other metadata here if needed
}

base_filename = os.path.splitext(os.path.basename(file_path))[0]
save_path = os.path.join(output_dir, f"{base_filename}_processed.npz")
np.savez(save_path, **save_dict)
print(f"Processed data and metadata saved to: {save_path}")

['#refs#', '#subsystem#', 'kgrid', 'sensor_data']
<HDF5 file "solid_liquid2.mat" (mode r)>


AttributeError: 'File' object has no attribute 'kgrid'

## 信号波形の可視化・解析
機械学習および信号波形の比較・解析をしやすいように変換した`.npz`ファイルを読み込み、可視化するスクリプトです。これら信号波形は画像形式に変換し解釈の精度を上げることもできます。一方で、一つのパルスの可視化にも意味があります。  
そこで、オプションに従いその画像化を行うスクリプトを`npz2png.py`という関数にまとめてあります。これらの引数として`full=True`ならば画像を返し、`full=False`ならば一パルス分の概形のみを表示します。  
## npz2png.py overall(util)
### input 
 - file_path
 - save_path
 - channel_index
 - start_time
 - end_time
 - full(=True or False)
 - pulse_index  
### output
 - *_"pulse or img".png
### 補足
画像を生成するスクリプトですが、これに関しては信号波形を直接画像化してもあまり意味はありません。そこで、ここでは検波処理（包絡線抽出）を行っています。。また、使うInterfaceの都合上、Nanの値があるとデータがすべて毀損されてしまうので、飽和を起こしている領域を0とする処理を行ってあります。また、画像化の際にtorchを用いGPUで処理することによって高速化を達成しています。一つの信号波形を表示する場合（full=False）はデバイス間の通信がボトルネックになりますので、データ転送は行わずCPUで処理しています。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import npz2png
# npz2img関数を実際に使って画像を保存してみる例
npz_file_path = "/home/smatsubara/documents/airlift/data/experiments/processed/P20240726-1600_processed.npz"
output_folder_path = "/home/smatsubara/documents/airlift/data/experiments/processed/"
npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=False, pulse_index=0)
npz2png(npz_file_path, output_folder_path, channel_index=1, start_time=0.0, end_time=None, full=True, pulse_index=0)


device: cuda
/home/smatsubara/documents/airlift/data/experiments/processed/P20240726-1600_processed_0img.png
/home/smatsubara/documents/airlift/data/experiments/processed/P20240726-1600_processed_0pulse.png
device: cuda
/home/smatsubara/documents/airlift/data/experiments/processed/P20240726-1600_processed_1img.png


In [ ]:
def find_dt_in_kgrid_fields(file_path):
    """
    kgridの他のフィールドからdtを探す
    """
    with h5py.File(file_path, 'r') as f:
        z_group = f['#refs#/z']
        
        print("=== kgridの全フィールドを確認 ===")
        
        # 各フィールドを確認
        for field_name in z_group.keys():
            field_obj = z_group[field_name]
            field_data = field_obj[()]
            
            print(f"\n{field_name}:")
            print(f"  型: {type(field_obj)}")
            print(f"  データ型: {field_obj.dtype}")
            print(f"  形状: {field_obj.shape}")
            print(f"  データ: {field_data}")
            
            # 数値フィールドでdtの可能性をチェック
            if hasattr(field_data, 'shape') and field_data.size == 1:
                val = field_data.item()
                if isinstance(val, (int, float, np.number)):
                    val_float = float(val)
                    if 1e-9 < val_float < 1e-3:  # 典型的なdtの範囲
                        print(f"  ✅ 潜在的なdt値: {val_float}")
                    elif val_float > 0:
                        print(f"  ℹ️  正の値: {val_float}")

# 実行
find_dt_in_kgrid_fields(file_path)